<a href="https://kritikseth.github.io/redirect" target="_parent"><img src="https://raw.githack.com/kritikseth/kritikseth/master/redirect.svg" alt="Kritik Seth"/></a>

In [ ]:
import pandas as pd
import numpy as np
from pandasql import sqldf

import math
from scipy import stats
from scipy.stats import geom
from sklearn.linear_model import LinearRegression
from statsmodels.stats.power import TTestIndPower, ttest_power

import re
from tqdm.notebook import tqdm

import seaborn as sns
from plotly import tools
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as py
import matplotlib.pyplot as plt
%matplotlib inline

from mpl_toolkits.mplot3d import Axes3D
from yellowbrick.classifier import ROCAUC

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category = DeprecationWarning)

R_STATE = 18714836 # random state

## Inference

In [ ]:
cpu_years = pd.read_excel('/content/gdrive/MyDrive/Capstone/cpu_names.xlsx')

In [ ]:
cpu_detail_cols = ['CPU Brand', 'CPU Model', 'CPU Series', 'CPU Generation', 'CPU Cores']

cpu_years['CPU Name'] = cpu_years['CPU Name'].apply(lambda x:x.replace('-', ' '))
# cpu_years.drop(['month'], axis=1, inplace=True)
cpu = pd.merge(df[cpu_detail_cols + ['CPU Name', 'CPU Transistors']], cpu_years, how='left', on='CPU Name')
cpu = cpu[~cpu['CPU Transistors'].isna() & ~cpu['CPU Generation'].isna()] #dropping na transistors

cpu.drop(['Unnamed: 3'], axis=1, inplace=True)
numeric_cols = ['CPU Transistors', 'CPU Generation']
cpu[numeric_cols] = cpu[numeric_cols].apply(pd.to_numeric)

cpu.dropna(inplace = True)
cpu = cpu.drop_duplicates()
cpu['Adjusted Transistors'] = cpu['CPU Transistors']/cpu['CPU Cores']
cpu['Adjusted Transistors'] = cpu['Adjusted Transistors'].apply(int)

cpu.sort_values('CPU Release Year')

temp = cpu.copy()
new_cols = [x.replace(' ', '_') for x in cpu.columns]
print(new_cols)
temp.columns = new_cols
temp.head()

In [ ]:
out = sqldf('''

    select max(Adjusted_Transistors) as Max_transistors, count(Adjusted_Transistors) as Models_released,
    avg(Adjusted_Transistors) as Avg_transistors,
    case when month < 1 then CPU_Release_Year - 1 
    when month > 9 then CPU_Release_Year + 1
    else CPU_Release_Year
    end as new_year
    from temp group by new_year

''')

In [ ]:
out['Double_Transistors'] = 2 * out['Max_transistors']
out['Actual_Transistors'] = out['Max_transistors'].shift(-2)

In [ ]:
out = out.replace(out.iloc[13][0], 1200) 
out = out.replace(out.iloc[6][0], 1303)
out = out.replace(out.iloc[7][0], 1303)
out = out.replace(out.iloc[11][0], 3100)

In [ ]:
out

In [ ]:
stat1, p1 = stats.ks_2samp(out['Actual_Transistors'][:-2], out['Double_Transistors'][:-2])

In [ ]:
stat2, p2 = stats.mannwhitneyu(out['Actual_Transistors'][:-2], out['Double_Transistors'][:-2])

In [ ]:
n1 = 12
n2 = 12
mu = (n1*n2)/2
sigma = math.sqrt((n1*n2)*(n1+n2+1)/12)
z = (stat2 - mu)/sigma
r = abs(z)/math.sqrt(n1)

In [ ]:
r

In [ ]:
kde_df = out[['Actual_Transistors', 'Double_Transistors']]

In [ ]:
ax = kde_df.plot.kde()
plt.xlabel('Transistors')
plt.ylabel('Density')
plt.title('Comparison of Distribution Between Actual Transistors and Double Transistors')
plt.savefig('KDE Plot Of Distributions')

In [ ]:
one = out['Double_Transistors'][:-2]
two = out['Actual_Transistors'][:-2]

In [ ]:
X = out['new_year'][:-2]
X_axis = np.arange(len(X))
plt.bar(X_axis - 0.2, one, 0.4, label = 'Double_Transistors')
plt.bar(X_axis + 0.2, two, 0.4, label = 'Actual_Transistors')
plt.xticks(X_axis, X)
plt.xlabel('Year')
plt.ylabel('Number of Transistors')
plt.title('Comparison of Distribution Between Actual Transistors and Double Transistors')
plt.xticks(rotation = 45)
plt.legend()
plt.show()
plt.savefig('Bar Plot Of Distributions')

In [ ]:
errorBar_df = out.copy()

In [ ]:
errorBar_df['Errors'] = abs(errorBar_df['Double_Transistors'] - errorBar_df['Actual_Transistors'])

In [ ]:
errorBar_df

In [ ]:
sem_Actual = out['Actual_Transistors'].std() / math.sqrt(12)
sem_Actual

In [ ]:
sem_Expected = out['Double_Transistors'].std() / math.sqrt(12)
sem_Expected

In [ ]:
fig, ax = plt.subplots(figsize = (10, 10))
ax.bar(errorBar_df['new_year'], errorBar_df['Actual_Transistors'],
       yerr=sem_Actual,
       align='center',
       alpha=0.5,
       ecolor='black',
       capsize=10)
plt.xlabel('Year')
plt.ylabel('Actual Number of Transistors')

In [ ]:
fig, ax = plt.subplots(figsize = (10, 10))
ax.bar(errorBar_df['new_year'], errorBar_df['Double_Transistors'],
       yerr=sem_Expected,
       align='center',
       alpha=0.5,
       ecolor='black',
       capsize=10)
plt.xlabel('Year')
plt.ylabel('Expected Number of Transistors')

## Data Imputation

In [ ]:
based_col = ['CPU Brand', 'CPU Model', 'CPU Series', 'CPU Generation']

geometric_cols = ['CPU Transistors']
linear_cols = ['CPU Die Size']
step_cols = []

# Geometric Imputation
if geometric_cols:
    for impute_col in geometric_cols:
        temp = df[based_col + [impute_col]].dropna(inplace=False)
        temp[impute_col] = pd.to_numeric(temp[impute_col])
        index = df[df[impute_col].isnull()].index.tolist()

        value = temp.groupby(based_col).agg('mean').reset_index().to_dict('split')['data']
        value_keys = [val[:4] for val in value]
        for ind in tqdm(index):

            key = tuple(df.loc[ind, based_col].values)
            previous = [val for val in value if key[:3] == tuple(val[:3])]
            if not previous:
                df.drop(ind, axis=0, inplace=True)
                continue
            gen = [int(p[3]) for p in previous]
            impute = [int(p[-1]) for p in previous]    

            if list(key) in value_keys:
                df.loc[ind, impute_col] = [val for val in value if key[:4] == tuple(val[:4])][0][-1]

            else:
                if int(key[3]) > min(gen) and int(key[3]) < max(gen):
                    geom = np.geomspace(impute[gen.index(min(gen))],
                                        impute[gen.index(max(gen))],
                                        max(gen) - min(gen) + 1)
                    df.loc[ind, impute_col] = geom[int(key[-1])-min(gen)]

                if int(key[3]) < min(gen):
                    gap = min(gen) - int(key[3])
                    val = impute[gen.index(min(gen))]
                    for i in range(gap): val /= 2
                    df.loc[ind, impute_col] = val
                
                else:
                    gap = int(key[3]) - max(gen)
                    val = impute[gen.index(min(gen))]
                    for i in range(gap): val *= 2
                    df.loc[ind, impute_col] = val


# Linear Imputation
if linear_cols:
    for impute_col in linear_cols:
        temp = df[based_col + [impute_col]].dropna(inplace=False)
        temp[impute_col] = pd.to_numeric(temp[impute_col])
        index = df[df[impute_col].isnull()].index.tolist()

        value = temp.groupby(based_col).agg('mean').reset_index().to_dict('split')['data']
        value_keys = [val[:4] for val in value]
        for ind in tqdm(index):

            key = tuple(df.loc[ind, based_col].values)
            previous = [val for val in value if key[:3] == tuple(val[:3])]
            if not previous:
                df.drop(ind, axis=0, inplace=True)
                continue
            gen = [int(p[3]) for p in previous]
            impute = [int(p[-1]) for p in previous]    

            if list(key) in value_keys:
                df.loc[ind, impute_col] = [val for val in value if key[:4] == tuple(val[:4])][0][-1]

            else:
                lr = LinearRegression()
                lr.fit(np.array(gen).reshape(-1, 1), np.array(impute).reshape(-1, 1))
                df.loc[ind, impute_col] = lr.predict([[7]])[0][0]


# Step Imputation
if step_cols:
    for impute_col in step_cols:
        temp = df[based_col + [impute_col]].dropna(inplace=False)
        temp[impute_col] = pd.to_numeric(temp[impute_col])
        index = df[df[impute_col].isnull()].index.tolist()

        value = temp.groupby(based_col).agg('mean').reset_index().to_dict('split')['data']
        value_keys = [val[:4] for val in value]
        for ind in tqdm(index):

            key = tuple(df.loc[ind, based_col].values)
            previous = [val for val in value if key[:3] == tuple(val[:3])]
            if not previous:
                df.drop(ind, axis=0, inplace=True)
                continue
            gen = [int(p[3]) for p in previous]
            impute = [int(p[-1]) for p in previous]    

            if list(key) in value_keys:
                df.loc[ind, impute_col] = [val for val in value if key[:4] == tuple(val[:4])][0][-1]

            else:
                steps = np.linspace(impute[gen.index(min(gen))],
                                    impute[gen.index(max(gen))],
                                    max(gen) - min(gen) + 1)
                step = step[1] - step[0]
                if int(key[3]) > min(gen) and int(key[3]) < max(gen):
                    df.loc[ind, impute_col] = steps[int(key[-1])-min(gen)]

                if int(key[3]) < min(gen):
                    gap = min(gen) - int(key[3])
                    val = impute[gen.index(min(gen))]
                    for i in range(gap): val -= step
                    df.loc[ind, impute_col] = val
                
                else:
                    gap = int(key[3]) - max(gen)
                    val = impute[gen.index(min(gen))]
                    for i in range(gap): val += step
                    df.loc[ind, impute_col] = val

In [ ]:
 missing_values_table(df)

<a href="https://kritikseth.github.io/redirect" target="_parent"><img src="https://raw.githack.com/kritikseth/kritikseth/master/redirect.svg" alt="Kritik Seth"/></a>